In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F 
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

spark = SparkSession.builder.appName("Jupyter").getOrCreate()

26/01/15 23:22:14 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
# Create a demo namespace and table (idempotent if the table already exists)
spark.sql("CREATE NAMESPACE IF NOT EXISTS iceberg.db")
spark.sql("CREATE TABLE IF NOT EXISTS iceberg.db.users (id BIGINT, name STRING, created_at TIMESTAMP) USING iceberg")
spark.sql("SELECT * FROM iceberg.db.users LIMIT 5").show()

+---+----+----------+
| id|name|created_at|
+---+----+----------+
+---+----+----------+



### 1. List the datasets

In [5]:
import os
os.getcwd()
os.listdir("/home/iceberg/data/")

['yellow_tripdata_2025-01.parquet', 'events.csv']

### 2. Load file with spark

In [4]:
events = spark.read.option("header", "true").csv("/home/iceberg/data/events.csv")
events.show()

+-----------+----------+--------+--------------------+----------+--------------------+
|    user_id| device_id|referrer|                host|       url|          event_time|
+-----------+----------+--------+--------------------+----------+--------------------+
| 1037710827| 532630305|    NULL| www.zachwilson.tech|         /|2021-03-08 17:27:...|
|  925588856| 532630305|    NULL|    www.eczachly.com|         /|2021-05-10 11:26:...|
|-1180485268| 532630305|    NULL|admin.zachwilson....|         /|2021-02-17 16:19:...|
|-1044833855| 532630305|    NULL| www.zachwilson.tech|         /|2021-09-24 15:53:...|
|  747494706| 532630305|    NULL| www.zachwilson.tech|         /|2021-09-26 16:03:...|
|  747494706| 532630305|    NULL|admin.zachwilson....|         /|2021-02-21 16:08:...|
| -824540328| 532630305|    NULL|admin.zachwilson....|         /|2021-09-28 17:23:...|
| -824540328| 532630305|    NULL|    www.eczachly.com|         /|2021-09-29 01:22:...|
| 1833036683| 532630305|    NULL|admin.zach

In [6]:
type(events)

pyspark.sql.dataframe.DataFrame

In [7]:
events.count()

404814

In [8]:
event1 = events

In [9]:
eve_df = events.join(events, on="device_id", how ="outer")
eve_df.count()

2180382254

In [10]:
df = spark.read.option("header", "true").parquet("/home/iceberg/data/yellow_tripdata_2025-01.parquet")
# df = spark.read.parquet("/home/iceberg/data/yellow_tripdata_2025-01.parquet")
# df = pd.read_parquet("/home/iceberg/data/yellow_tripdata_2025-01.parquet")

In [11]:
df.show(5, truncate=False)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|1       |2025-01-01 00:18:38 |2025-01-01 00:26:59  |1              |1.6          |1         |N                 |229         |237 

In [12]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)
 |-- cbd_congestion_fee: double (nullable = true)



In [9]:
df.count()

3475226

In [13]:
df_gp = df.groupBy("VendorId").count()
df_gp.show(8, truncate=False)

+--------+-------+
|VendorId|count  |
+--------+-------+
|1       |753671 |
|7       |1206   |
|2       |2719860|
|6       |489    |
+--------+-------+



In [11]:
df1 =df 

In [27]:
df2 = df.join(df, on = "VendorId", how = "left")

# df2 = df.join(F.broadcast(df_gp), on = "VendorId", how = "left")

In [ ]:
print(df.count())
print(df2.count())

3475226


[Stage 73:=============================>                            (4 + 4) / 8]

In [ ]:
df